Dhananjay Tiwari,
Ph.D. Mechanical Engineering,
Mechanical Engineering Laboratory, 1204
University of Illinois at Urbana Champaign

This file has the code for implementation of FLPO optimization problem using using scipy optimizers

In [ ]:
# include libraries and dependencies
import numpy as np # eh! computational python programming and not include this? Can't happen!

In [ ]:
# define the structure of FLPO problem
N = 1 # number of nodes
M = 3 # number of facilities


In [1]:
# create FLPO class and define its associated functions
class FLPO():

    # define data members of the class
    n : int # number of nodes
    f : int # number of facilities
    M : int # number of stages
    node_id : list # node indices
    fac_id : list # facility indices
    dest_id : list # destination id
    fac_param : dict # facility parameters

    def __init__(self, n, f, M):
        # this function intializes the FLPO structure with assigned values

        self.n = n # initialize the number of nodes
        self.f = f # initialize the number of facilities

    def distortion(pb_asn, fac_param): # depends probability associations and parameters of facilities
        return 0
    
    def entropy(pb_asn): # depends on probability associations
        return 0
    
    def free_energy():
        return 0

